# 📊 Car Sales Data Analysis - SQL Project

## Project Overview

This project analyzes a relational database (stores.db) that contains sales transactions, customers, products, employees, and payments for a car sales business. The goal is to extract meaningful insights using SQL queries and showcase database design principles. Specifically, this project aims to answer the following key business questions:

1. Which products should we order more of or less of?
2.  How should we tailor marketing and communication strategies to customer behaviors?
3. How much can we spend on acquiring new customers?

To ensure better documentation and reproducibility, I will be using SQLite and Pandas from Python, allowing for efficient query execution, data analysis, and structured reporting.

In [27]:
# Importing necessary packages
import sqlite3
import pandas as pd

Establishing the connection to the database.

In [28]:
# Path to your SQLite database
db_path = "/Users/aravindshrenivas/Downloads/sales_car_data_analysis/stores.db"

# Connect to the SQLite database 
conn = sqlite3.connect(db_path)

print("Connected to Database!")


Connected to Database!


## 📂 Database Schema and Relationships

First let's write a query to view the tables in the database.

In [29]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""

df = pd.read_sql_query(query, conn)
df


,name
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


From the above output, We can see that the database consists of 8 tables.

### Conceptual Data Model

The CDM is designed using crow's foot notation.

![alt text](./cdm.png)

### 🗄️ Entity-Relationship Breakdown

| **Entity**      | **Primary Key**      | **Related Entities**                     | **Relationship Type**   |
|---------------|--------------------|--------------------------------|----------------|
| `customers`   | `customerNumber`   | `orders`, `payments`          | One-to-Many   |
| `employees`   | `employeeNumber`   | `customers`, `offices`        | One-to-Many   |
| `offices`     | `officeCode`       | `employees`                   | One-to-Many   |
| `orders`      | `orderNumber`      | `orderdetails`, `customers`   | One-to-Many   |
| `orderdetails`| (`orderNumber`, `productCode`) | `orders`, `products` | Many-to-Many   |
| `products`    | `productCode`      | `productlines`, `orderdetails`| Many-to-One   |
| `payments`    | `checkNumber`      | `customers`                   | One-to-Many   |
| `productlines`| `productLine`      | `products`                    | One-to-Many   |

### Primary and Foreign keys

In [30]:
# Query to extract Primary Keys
query_pk = """
SELECT m.name AS table_name, 
       p.name AS column_name,
       'PRIMARY KEY' AS constraint_type
FROM sqlite_master AS m
JOIN pragma_table_info(m.name) AS p
ON p.pk = 1
WHERE m.type = 'table';
"""

# Execute Primary Key query
df_pk = pd.read_sql_query(query_pk, conn)

# Fetch Foreign Keys for each table separately
foreign_keys = []
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

for table in tables['name']:
    query_fk = f"PRAGMA foreign_key_list({table})"
    df_fk = pd.read_sql_query(query_fk, conn)
    
    for _, row in df_fk.iterrows():
        foreign_keys.append({
            "table_name": table,
            "column_name": row['from'],
            "constraint_type": f"FOREIGN KEY → {row['table']}({row['to']})"
        })

# Convert foreign keys data to DataFrame
df_fk = pd.DataFrame(foreign_keys)

# Combine both Primary Key and Foreign Key data
df_constraints = pd.concat([df_pk, df_fk], ignore_index=True)

print("\nDatabase Table Relationships:")
df_constraints


Database Table Relationships:


,table_name,column_name,constraint_type
0,customers,customerNumber,PRIMARY KEY
1,employees,employeeNumber,PRIMARY KEY
2,offices,officeCode,PRIMARY KEY
3,orderdetails,orderNumber,PRIMARY KEY
4,orders,orderNumber,PRIMARY KEY
5,payments,customerNumber,PRIMARY KEY
6,productlines,productLine,PRIMARY KEY
7,products,productCode,PRIMARY KEY
8,customers,salesRepEmployeeNumber,FOREIGN KEY → employees(employeeNumber)
9,employees,officeCode,FOREIGN KEY → offices(officeCode)


### Table Summary

The following SQL query provides an overview of all tables, including the number of attributes (columns) and total records per table.

In [31]:
# Query to get table details
query = """
SELECT 'customers' AS table_name, 
       (SELECT COUNT(*) FROM pragma_table_info('customers')) AS num_attributes, 
       (SELECT COUNT(*) FROM customers) AS num_rows
UNION ALL
SELECT 'employees', 
       (SELECT COUNT(*) FROM pragma_table_info('employees')), 
       (SELECT COUNT(*) FROM employees)
UNION ALL
SELECT 'offices', 
       (SELECT COUNT(*) FROM pragma_table_info('offices')), 
       (SELECT COUNT(*) FROM offices)
UNION ALL
SELECT 'orderdetails', 
       (SELECT COUNT(*) FROM pragma_table_info('orderdetails')), 
       (SELECT COUNT(*) FROM orderdetails)
UNION ALL
SELECT 'orders', 
       (SELECT COUNT(*) FROM pragma_table_info('orders')), 
       (SELECT COUNT(*) FROM orders)
UNION ALL
SELECT 'payments', 
       (SELECT COUNT(*) FROM pragma_table_info('payments')), 
       (SELECT COUNT(*) FROM payments)
UNION ALL
SELECT 'productlines', 
       (SELECT COUNT(*) FROM pragma_table_info('productlines')), 
       (SELECT COUNT(*) FROM productlines)
UNION ALL
SELECT 'products', 
       (SELECT COUNT(*) FROM pragma_table_info('products')), 
       (SELECT COUNT(*) FROM products);
"""

df = pd.read_sql_query(query, conn)

df


,table_name,num_attributes,num_rows
0,customers,13,122
1,employees,8,23
2,offices,9,7
3,orderdetails,5,2996
4,orders,7,326
5,payments,4,273
6,productlines,4,7
7,products,9,110


## 📊 Question 1: Which products should we order more of or less of?

### 🔍 Understanding the Question
This question helps us identify:
- **Low stock products** that are close to running out.
- **Best-selling products** that generate the most revenue.
- **Priority restocking items** that need immediate attention.

By combining **low stock data** with **sales performance**, we ensure that:
- Popular products do not go out of stock.
- Inventory is managed efficiently to meet customer demand.


#### Step 1: Identify Low Stock Products

A product is considered **low stock** if its ordered quantity is high compared to its available stock.  

We will select **the top 10 products** that are **most at risk of running out**.

To achieve this, we:
- Use the `orderdetails` table to sum `quantityOrdered` for each product.
- Divide this by `quantityInStock` from the `products` table.
- Select the **Top 10 products** with the highest depletion rate.

In [32]:
# Query to compute low stock products
query_low_stock = """
WITH low_stock_table AS (
    SELECT productCode, 
           ROUND(SUM(quantityOrdered) * 1.0 / 
           (SELECT quantityInStock FROM products AS p WHERE p.productCode = o.productCode), 2) 
           AS low_stock
    FROM orderdetails AS o 
    GROUP BY productCode
    ORDER BY low_stock DESC
    LIMIT 10
)
SELECT * FROM low_stock_table;
"""

df_low_stock = pd.read_sql_query(query_low_stock, conn)

df_low_stock


,productCode,low_stock
0,S24_2000,67.67
1,S12_1099,13.72
2,S32_4289,7.15
3,S32_1374,5.70
4,S72_3212,2.31
5,S700_3167,1.90
6,S50_4713,1.65
7,S18_2795,1.61
8,S18_2248,1.54
9,S700_1938,1.22


Inference:

- This table highlights products that are at risk of stock depletion based on high demand but low inventory levels.
- A high low-stock ratio means the product is selling quickly and may go out of stock soon.
- A low low-stock ratio means the product still has sufficient inventory despite sales.

Business Insights:

- Products with a high low-stock ratio should be prioritized for immediate restocking to avoid lost sales.
- Inventory managers can use this data to adjust reorder levels and prevent supply chain disruptions.

#### Step 2: Identify High-Performing Products

A product's **performance** is measured by its **total sales revenue**.  
We will select **the top 10 highest revenue-generating products**.

To achieve this, we:
- Multiply `quantityOrdered` by `priceEach` from the `orderdetails` table.
- Sum the total revenue for each product.
- Select the **Top 10 highest-performing products**.

In [33]:
# Query to compute product performance
query_product_performance = """
WITH product_performance_table AS (
    SELECT productCode, 
           SUM(quantityOrdered * priceEach) AS product_performance
    FROM orderdetails
    GROUP BY productCode
    ORDER BY product_performance DESC
    LIMIT 10
)
SELECT * FROM product_performance_table;
"""

df_product_performance = pd.read_sql_query(query_product_performance, conn)

df_product_performance


,productCode,product_performance
0,S18_3232,276839.98
1,S12_1108,190755.86
2,S10_1949,190017.96
3,S10_4698,170686.00
4,S12_1099,161531.48
5,S12_3891,152543.02
6,S18_1662,144959.91
7,S18_2238,142530.63
8,S18_1749,140535.60
9,S12_2823,135767.03


Inference:

- This table shows best-performing products based on total revenue generated.
- Products with a high product performance score contribute significantly to business revenue.
- If a product has low revenue, it might not be popular or priced effectively.

Business Insights:

- High-performing products should be promoted more aggressively through marketing campaigns.
- Low-performing products may require price adjustments, bundling, or discontinuation if they do not contribute much to overall revenue.

#### Step 3: Prioritize Restocking Based on Demand

To determine which products need urgent restocking, we:
1. Identify **products that are low in stock**.
2. Identify **products that generate the highest revenue**.
3. Select products that **appear in both categories**.

This ensures that we prioritize **fast-selling products** that are at risk of stockouts.


In [34]:
# Query to prioritize restocking products
query_priority_restocking = """
WITH 
low_stock_table AS (
    SELECT productCode 
    FROM orderdetails AS o 
    GROUP BY productCode
    ORDER BY SUM(quantityOrdered) * 1.0 / 
             (SELECT quantityInStock FROM products AS p WHERE p.productCode = o.productCode) DESC
    LIMIT 10
),
product_to_restock AS (
    SELECT productCode, SUM(quantityOrdered * priceEach) AS product_performance
    FROM orderdetails
    WHERE productCode IN (SELECT productCode FROM low_stock_table)
    GROUP BY productCode
    ORDER BY product_performance DESC
    LIMIT 10
)
SELECT p.productCode, p.productName, p.productLine
FROM products p
WHERE p.productCode IN (SELECT productCode FROM product_to_restock);
"""

df_priority_restocking = pd.read_sql_query(query_priority_restocking, conn)

df_priority_restocking


,productCode,productName,productLine
0,S12_1099,1968 Ford Mustang,Classic Cars
1,S18_2248,1911 Ford Town Car,Vintage Cars
2,S18_2795,1928 Mercedes-Benz SSK,Vintage Cars
3,S24_2000,1960 BSA Gold Star DBD34,Motorcycles
4,S32_1374,1997 BMW F650 ST,Motorcycles
5,S32_4289,1928 Ford Phaeton Deluxe,Vintage Cars
6,S50_4713,2002 Yamaha YZR M1,Motorcycles
7,S700_1938,The Mayflower,Ships
8,S700_3167,F/A 18 Hornet 1/72,Planes
9,S72_3212,Pont Yacht,Ships


Inference:

- This table combines the previous insights to show which products are both high-selling and at risk of going out of stock.
- If a product appears here, it means customers want it, but supply may not meet demand.

Business Insights:

- These products should be immediately restocked to maximize sales opportunities.
- Businesses can work with suppliers to negotiate faster deliveries for these critical items.
- Adjusting safety stock levels can prevent frequent shortages of high-demand products.

## 📊 Question 2: How Should We Match Marketing and Communication Strategies to Customer Behavior?

### Understanding the Question
This analysis helps identify:
- **VIP Customers** – The most profitable customers for the store.
- **Less Engaged Customers** – Customers who contribute the least profit.

Understanding these groups will help us:
- **Reward VIP customers** with loyalty programs, exclusive discounts, and premium support.
- **Re-engage less-engaged customers** with targeted promotions, personalized emails, and special offers.

We will analyze customer profitability by:
1. Calculating **profit per customer** using purchase data.  
2. Identifying **Top 5 VIP Customers** based on profit.  
3. Identifying **Bottom 5 Less Engaged Customers** based on profit.  
4. Computing **Average Customer Lifetime Value (LTV)** to assess overall profitability.


Step 1: Compute Profit Per Customer

To determine **customer profitability**, we:
1. **Join** the `orders`, `orderdetails`, and `products` tables.  
2. For each customer, the profit, which is the sum of quantityOrdered multiplied by priceEach minus buyPrice: SUM(quantityOrdered * (priceEach - buyPrice)).
3. **Group results by `customerNumber`** to find total profit per customer.


In [35]:
# Query to compute total profit per customer
query_profit_per_customer = """
WITH profit_per_customer_table AS (
    SELECT o.customerNumber, 
           SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit
    FROM orders AS o
    JOIN orderdetails AS od ON od.orderNumber = o.orderNumber
    JOIN products AS p ON p.productCode = od.productCode
    GROUP BY o.customerNumber
)
SELECT * FROM profit_per_customer_table
ORDER BY total_profit DESC;
"""

df_profit_per_customer = pd.read_sql_query(query_profit_per_customer, conn)

df_profit_per_customer


,customerNumber,total_profit
0,141,326519.66
1,124,236769.39
2,151,72370.09
3,114,70311.07
4,119,60875.30
...,...,...
93,489,10868.04
94,103,10063.80
95,473,9532.93
96,198,6586.02


Inference:
- Customers with **high profit** should be prioritized for **VIP engagement strategies**.
- Customers with **low profit** may need **personalized marketing** to increase spending.

Step 2: Identify VIP Customers

We will extract the **Top 5 most profitable customers** based on total profit.


In [36]:
# Query to identify top 5 VIP customers
query_vip_customers = """
WITH profit_per_customer_table AS (
    SELECT o.customerNumber, 
           SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit
    FROM orders AS o
    JOIN orderdetails AS od ON od.orderNumber = o.orderNumber
    JOIN products AS p ON p.productCode = od.productCode
    GROUP BY o.customerNumber
)
SELECT c.customerNumber, c.contactLastName, c.contactFirstName, c.city, c.country, ppct.total_profit
FROM customers AS c
JOIN profit_per_customer_table AS ppct ON c.customerNumber = ppct.customerNumber
ORDER BY ppct.total_profit DESC
LIMIT 5;
"""

df_vip_customers = pd.read_sql_query(query_vip_customers, conn)

df_vip_customers


,customerNumber,contactLastName,contactFirstName,city,country,total_profit
0,141,Freyre,Diego,Madrid,Spain,326519.66
1,124,Nelson,Susan,San Rafael,USA,236769.39
2,151,Young,Jeff,NYC,USA,72370.09
3,114,Ferguson,Peter,Melbourne,Australia,70311.07
4,119,Labrune,Janine,Nantes,France,60875.30



Inference:
- These customers generate **the highest revenue for the business**.
- The store should **offer loyalty benefits**, such as discounts, early access to new products, or exclusive customer support.

Step 3: Identify Less Engaged Customers

We will extract the **Bottom 5 customers** who bring in the least profit.


In [37]:
# Query to identify bottom 5 less-engaged customers
query_less_engaged_customers = """
WITH profit_per_customer_table AS (
    SELECT o.customerNumber, 
           SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit
    FROM orders AS o
    JOIN orderdetails AS od ON od.orderNumber = o.orderNumber
    JOIN products AS p ON p.productCode = od.productCode
    GROUP BY o.customerNumber
)
SELECT c.customerNumber, c.contactLastName, c.contactFirstName, c.city, c.country, ppct.total_profit
FROM customers AS c
JOIN profit_per_customer_table AS ppct ON c.customerNumber = ppct.customerNumber
ORDER BY ppct.total_profit 
LIMIT 5;
"""

df_less_engaged_customers = pd.read_sql_query(query_less_engaged_customers, conn)

df_less_engaged_customers


,customerNumber,contactLastName,contactFirstName,city,country,total_profit
0,219,Young,Mary,Glendale,USA,2610.87
1,198,Taylor,Leslie,Brickhaven,USA,6586.02
2,473,Ricotti,Franco,Milan,Italy,9532.93
3,103,Schmitt,Carine,Nantes,France,10063.80
4,489,Smith,Thomas,London,UK,10868.04



Inference:
- These customers **do not spend much** at the store.
- We can implement **targeted marketing campaigns** such as:
  - **Discount offers** to encourage more purchases.
  - **Personalized emails** highlighting products based on their past purchases.
  - **Limited-time promotions** to re-engage inactive customers.

Step 4: Compute Average Customer Lifetime Value (LTV)

Customer Lifetime Value (LTV) represents the **average profit per customer**.

In [38]:
# Query to compute average Customer Lifetime Value (LTV)
query_ltv = """
WITH profit_per_customer_table AS (
    SELECT o.customerNumber, 
           SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit
    FROM orders AS o
    JOIN orderdetails AS od ON od.orderNumber = o.orderNumber
    JOIN products AS p ON p.productCode = od.productCode
    GROUP BY o.customerNumber
)
SELECT AVG(total_profit) AS avg_customer_lifetime_value
FROM profit_per_customer_table;
"""

# Execute the query
df_ltv = pd.read_sql_query(query_ltv, conn)

# Display results
display(df_ltv)


,avg_customer_lifetime_value
0,39039.594388


Inference:
- LTV tells us how much profit an average customer generates during their lifetime with our store. We can use it to predict our future profit. So, if we get ten new customers next month, we'll earn 390,395 dollars, and we can decide based on this prediction how much we can spend on acquiring new customers.

## 📊 Question 3: How Much Can We Spend on Acquiring New Customers?

### Understanding the Question
To determine **how much we can spend on acquiring new customers**, we need to calculate:
1. **Customer Acquisition Cost (CAC)** – The cost incurred to acquire a new customer.  
2. **LTV to CAC Ratio** – A key indicator of **marketing efficiency** and **customer profitability**.

This will help us decide **whether our current acquisition strategy is sustainable** and **how much we should invest in acquiring new customers**.


Step 1: Compute Customer Acquisition Cost (CAC)

In [39]:
# Assumed Total Marketing Cost for a specific period
TotalMarketingCost = 100000  # Example marketing budget in dollars

# Query to compute number of new customers acquired in the last year
query_new_customers = """
SELECT COUNT(DISTINCT customerNumber) AS new_customers_acquired
FROM customers
WHERE customerNumber NOT IN (
    SELECT customerNumber FROM orders WHERE orderDate < DATE('now', '-1 year')
);
"""

df_new_customers = pd.read_sql_query(query_new_customers, conn)

# Compute CAC
new_customers_acquired = df_new_customers.iloc[0, 0]  # Extract value from dataframe
CAC = round(TotalMarketingCost / new_customers_acquired, 2) if new_customers_acquired > 0 else 0

# Display CAC value
print(f"Customer Acquisition Cost (CAC): ${CAC} per new customer")


Customer Acquisition Cost (CAC): $4166.67 per new customer


Inference:
- If **CAC is lower than LTV**, acquiring new customers is **profitable**.
- If **CAC is higher than LTV**, marketing spending **needs to be optimized**.

Step 2: Compute LTV to CAC Ratio

In [40]:
# Compute average Customer LTV
query_avg_ltv = """
SELECT ROUND(AVG(customer_LTV), 2) AS avg_ltv FROM (
    WITH profit_per_customer AS (
        SELECT o.customerNumber, 
               SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS total_profit,
               COUNT(DISTINCT o.orderNumber) AS total_orders
        FROM orders AS o
        JOIN orderdetails AS od ON od.orderNumber = o.orderNumber
        JOIN products AS p ON p.productCode = od.productCode
        GROUP BY o.customerNumber
    )
    SELECT ROUND(total_profit / total_orders, 2) AS customer_LTV FROM profit_per_customer
);
"""

# Execute the query
df_avg_ltv = pd.read_sql_query(query_avg_ltv, conn)
average_LTV = df_avg_ltv.iloc[0, 0]  # Extract value from dataframe

# Compute LTV to CAC Ratio
ltv_to_cac_ratio = round(average_LTV / CAC, 2) if CAC > 0 else 0

# Display LTV to CAC Ratio
print(f"LTV to CAC Ratio: {ltv_to_cac_ratio}")


LTV to CAC Ratio: 2.78


Interpreting the Ratio:
- **LTV to CAC > 3** → Marketing is efficient; we can **spend more on acquisition**.
- **LTV to CAC ≈ 1-3** → Acquisition is **moderately profitable**; we should **optimize retention strategies**.
- **LTV to CAC < 1** → Acquisition costs **exceed customer value**, leading to **losses**.

A higher ratio means **acquiring new customers is cost-effective**.

## Conclusion

- Vintage cars and motorcycles are the priority for restocking. They sell frequently, and they are the highest-performance products.
- The VIP and non-VIP lists denote our most loyal customers and least engaged customers. Now that we have identified them, we can strategize on enhancing customer loyalty and attracting new customers effectively.
- LTV indicates the total profit an average customer generates over their lifetime with our store, helping us forecast future earnings. This prediction enables us to determine how much we can invest in acquiring new customers.